## Metadata Transformation for Submissions to Single Cell Portal (SCP) and CellxGene

#### Author: Hannah Kang

##### Study Information: 
The aim of this study is to understand the many types of neurons that carry out information processing in the brain evolved recently by focusing on the retina, the thin film of neurons in the eye that initiates vision. Our approach entails high-throughput single-cell RNA-seq where gene expression is quantified in hundreds of thousands of single retinal neurons, and computational methods are used to process and integrate these datasets. We have recently completed a project that involves generating and integrating single-cell RNA-seq data across an unprecedented 17 species.

Date Last Modified: 2/16/2024 (Before 11/18/2023)


In [1]:
# Import Statements
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata
import h5py
import pymart as pm

from pybiomart import Dataset

In [2]:
# Reference data from Professor Shekhar

# data = pd.read_csv("rgc_metadata_eg.txt")
# data.head(10)

In [3]:
# Marmosets

file_path = "/Users/hannahkang/Desktop/shekhar-lab/Original Files/MarmosetFovea_BC_count_mat.csv"

countmatrix_marmoset = pd.read_csv(file_path)
countmatrix_marmoset

,Unnamed: 0,possorted_genome_bam_EXKMG:AAAGATGAGCCAGGATx,possorted_genome_bam_EXKMG:AAAGATGAGGCCCTCAx,possorted_genome_bam_EXKMG:AAACGGGTCTTGTATCx,possorted_genome_bam_EXKMG:AAATGCCCACTAGTACx,possorted_genome_bam_EXKMG:AAACGGGGTCCTAGCGx,possorted_genome_bam_EXKMG:AAAGATGAGTGATCGGx,possorted_genome_bam_EXKMG:AAATGCCCATCCTTGCx,possorted_genome_bam_EXKMG:AAAGCAAGTTGAACTCx,possorted_genome_bam_EXKMG:AAAGATGTCGCCAAATx,...,possorted_genome_bam_K564P:TTTATGCGTGCAACTTx,possorted_genome_bam_K564P:TTTGGTTCACCTATCCx,possorted_genome_bam_K564P:TTTACTGGTTCCTCCAx,possorted_genome_bam_K564P:TTTCCTCGTCAACATCx,possorted_genome_bam_K564P:TTTATGCGTCTTTCATx,possorted_genome_bam_K564P:TTTGGTTGTTGAGTTCx,possorted_genome_bam_K564P:TTTATGCCATGTTCCCx,possorted_genome_bam_K564P:TTTATGCCAATCACACx,possorted_genome_bam_K564P:TTTGTCATCTGTTTGTx,possorted_genome_bam_K564P:TTTATGCCACATGTGTx
0,LOC108590668,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,SLITRK6,2,0,0,2,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,LOC103791423,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,LOC108590671,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,LOC108588276,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27660,LOC108589989,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27661,LOC103790416,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27662,LOC108589990,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27663,VAMP7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Metadata Transformation for Marmoset Dataset

In [4]:
file_path = "/Users/hannahkang/Desktop/shekhar-lab/Original Files/MarmosetFovea_BC_metadata.csv"

metadata_marmoset = pd.read_csv(file_path)
metadata_marmoset

,Unnamed: 0,orig.ident,nCount_RNA,nFeature_RNA,orig.file,animal,RNA_snn_res.0.5,seurat_clusters,dendro_order,cell_class,integrated_snn_res.1,integrated_snn_res.0.8,all_BC_labels,labels,type,Mammal_OT,Nonmammal_OT
0,possorted_genome_bam_EXKMG:AAAGATGAGCCAGGATx,possorted_genome_bam_EXKMG,1730,1194,Marmoset1FoveaS1,1,0,2,0,BC,1,1,1,2,f2,BC1A,absent
1,possorted_genome_bam_EXKMG:AAAGATGAGGCCCTCAx,possorted_genome_bam_EXKMG,3341,2034,Marmoset1FoveaS1,1,2,1,1,BC,0,0,2,1,f1,absent,absent
2,possorted_genome_bam_EXKMG:AAACGGGTCTTGTATCx,possorted_genome_bam_EXKMG,2549,1519,Marmoset1FoveaS1,1,2,1,1,BC,0,0,2,1,f1,absent,absent
3,possorted_genome_bam_EXKMG:AAATGCCCACTAGTACx,possorted_genome_bam_EXKMG,2145,1471,Marmoset1FoveaS1,1,0,2,0,BC,1,1,1,2,f2,absent,absent
4,possorted_genome_bam_EXKMG:AAACGGGGTCCTAGCGx,possorted_genome_bam_EXKMG,1445,997,Marmoset1FoveaS1,1,8,4,3,BC,3,3,4,4,f4,BC4,absent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12551,possorted_genome_bam_K564P:TTTGGTTGTTGAGTTCx,possorted_genome_bam_K564P,1709,1169,AdultmarmosetFovea2S2,2,9,5,4,BC,4,4,5,5,f5,absent,absent
12552,possorted_genome_bam_K564P:TTTATGCCATGTTCCCx,possorted_genome_bam_K564P,1910,1351,AdultmarmosetFovea2S2,2,0,2,0,BC,9,6,1,2,f2,BC1A,absent
12553,possorted_genome_bam_K564P:TTTATGCCAATCACACx,possorted_genome_bam_K564P,2292,1548,AdultmarmosetFovea2S2,2,0,2,0,BC,1,1,1,2,f2,absent,absent
12554,possorted_genome_bam_K564P:TTTGTCATCTGTTTGTx,possorted_genome_bam_K564P,2963,1794,AdultmarmosetFovea2S2,2,2,1,1,BC,0,0,2,1,f1,absent,absent


In [5]:
# Remove the prefix from the "Unnamed: 0" column
metadata_marmoset['Unnamed: 0'] = metadata_marmoset['Unnamed: 0'].str.replace('possorted_genome_bam_', '') 

metadata_marmoset = metadata_marmoset.rename(columns={'Unnamed: 0': 'NAME'})
metadata_marmoset['NAME'] = metadata_marmoset['NAME'].str.replace(':', '_')

# Define column values
column_values = {
    'biosample_id': metadata_marmoset['orig.file'],
    'donor_id': metadata_marmoset['animal'],
    'species': 'NCBITAXON_9483',
    'species__ontology_label': 'Callithrix jacchus',
    'disease': 'PATO_0000461',
    'disease__ontology_label': 'normal',
    'organ': 'UBERON_0000966',
    'organ__ontology_label': 'retina',
    'library_preparation_protocol': 'EFO_0009899',
    'library_preparation_protocol__ontology_label': "10X 3' v2",
    'sex': 'unknown'
}

# Insert columns into DataFrame
for i, (col_name, col_values) in enumerate(column_values.items(), start=1):
    metadata_marmoset.insert(i, col_name, col_values)

# Add 'TYPE' row
group_row = pd.Series(['TYPE'] + ['group'] * (len(metadata_marmoset.columns) - 1), index=metadata_marmoset.columns)
metadata_marmoset = pd.concat([group_row.to_frame().T, metadata_marmoset], ignore_index=True)

# Limit DataFrame to the first 12 columns
metadata_marmoset = metadata_marmoset.iloc[:, :12]
metadata_marmoset


,NAME,biosample_id,donor_id,species,species__ontology_label,disease,disease__ontology_label,organ,organ__ontology_label,library_preparation_protocol,library_preparation_protocol__ontology_label,sex
0,TYPE,group,group,group,group,group,group,group,group,group,group,group
1,EXKMG_AAAGATGAGCCAGGATx,Marmoset1FoveaS1,1,NCBITAXON_9483,Callithrix jacchus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
2,EXKMG_AAAGATGAGGCCCTCAx,Marmoset1FoveaS1,1,NCBITAXON_9483,Callithrix jacchus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
3,EXKMG_AAACGGGTCTTGTATCx,Marmoset1FoveaS1,1,NCBITAXON_9483,Callithrix jacchus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
4,EXKMG_AAATGCCCACTAGTACx,Marmoset1FoveaS1,1,NCBITAXON_9483,Callithrix jacchus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...
12552,K564P_TTTGGTTGTTGAGTTCx,AdultmarmosetFovea2S2,2,NCBITAXON_9483,Callithrix jacchus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
12553,K564P_TTTATGCCATGTTCCCx,AdultmarmosetFovea2S2,2,NCBITAXON_9483,Callithrix jacchus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
12554,K564P_TTTATGCCAATCACACx,AdultmarmosetFovea2S2,2,NCBITAXON_9483,Callithrix jacchus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
12555,K564P_TTTGTCATCTGTTTGTx,AdultmarmosetFovea2S2,2,NCBITAXON_9483,Callithrix jacchus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown


### Metadata Transformation for Mouse Dataset

In [6]:
file_path = "/Users/hannahkang/Desktop/shekhar-lab/Original Files/Mouse_BC_metadata.csv"

metadata_mouse = pd.read_csv(file_path)
metadata_mouse

,Unnamed: 0,orig.ident,nCount_RNA,nFeature_RNA,orig.file,animal,RNA_snn_res.0.5,seurat_clusters,dendro_order,integrated_snn_res.0.8,integrated_snn_res.0.5,barcode,annotated,type,Mammal_OT,Nonmammal_OT
0,possorted_genome_bam_Z0OYT:AAACGGGCAACACGCCx,possorted_genome_bam_Z0OYT,4424.0,2223.0,CTRLC57AllOther1,1.0,8.0,10.0,8.0,14.0,11.0,oncBC_CtC57AllOtherR1_AAACGGGCAACACGCC-1,BC5A,BC5A,absent,BC8/9
1,possorted_genome_bam_Z0OYT:AAACCTGAGTGTACGGx,possorted_genome_bam_Z0OYT,5860.0,2708.0,CTRLC57AllOther1,1.0,10.0,8.0,10.0,11.0,9.0,oncBC_CtC57AllOtherR1_AAACCTGAGTGTACGG-1,BC4,BC4,BC4,BC4
2,possorted_genome_bam_Z0OYT:AAAGTAGGTTGTTTGGx,possorted_genome_bam_Z0OYT,4458.0,2149.0,CTRLC57AllOther1,1.0,13.0,1.0,13.0,0.0,0.0,oncBC_CtC57AllOtherR1_AAAGTAGGTTGTTTGG-1,RBC,RBC,absent,absent
3,possorted_genome_bam_Z0OYT:AAAGTAGCATTCCTCGx,possorted_genome_bam_Z0OYT,1841.0,1025.0,CTRLC57AllOther1,1.0,11.0,1.0,11.0,1.0,2.0,oncBC_CtC57AllOtherR1_AAAGTAGCATTCCTCG-1,RBC,RBC,absent,RBC
4,possorted_genome_bam_Z0OYT:AAATGCCAGCCACTATx,possorted_genome_bam_Z0OYT,1447.0,990.0,CTRLC57AllOther1,1.0,40.0,15.0,40.0,20.0,18.0,oncBC_CtC57AllOtherR1_AAATGCCAGCCACTAT-1,BC1B,BC1B,BC2,BC1B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8146,Mouse;possorted_genome_bam_H5JEB:TTTACTGTCGGCA...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,BC2
8147,Mouse;possorted_genome_bam_H5JEB:TTTGGTTGTTAAA...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,BC3B
8148,Mouse;possorted_genome_bam_H5JEB:TTTATGCGTCTGA...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,BC5A
8149,Mouse;possorted_genome_bam_H5JEB:TTTCCTCGTTCAG...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,BC3B


In [7]:
# Remove the prefix from the "Unnamed: 0" column
metadata_mouse.loc[:, 'Unnamed: 0'] = metadata_mouse['Unnamed: 0'].str.replace('Mouse;possorted_genome_bam_', '')
metadata_mouse.loc[:, 'Unnamed: 0'] = metadata_mouse['Unnamed: 0'].str.replace('possorted_genome_bam_', '')

metadata_mouse = metadata_mouse.rename(columns={'Unnamed: 0': 'NAME'})
metadata_mouse['NAME'] = metadata_mouse['NAME'].str.replace(':', '_')

# Define column values
column_values = {
    'biosample_id': metadata_mouse['orig.file'],
    'donor_id': metadata_mouse['animal'],
    'species': 'NCBITaxon_10090',
    'species__ontology_label': 'Mus musculus',
    'disease': 'PATO_0000461',
    'disease__ontology_label': 'normal',
    'organ': 'UBERON_0000966',
    'organ__ontology_label': 'retina',
    'library_preparation_protocol': 'EFO_0009899',
    'library_preparation_protocol__ontology_label': "10X 3' v2",
    'sex': 'unknown'
}

# Insert columns into DataFrame
for i, (col_name, col_values) in enumerate(column_values.items(), start=1):
    metadata_mouse.insert(i, col_name, col_values)

# Add 'TYPE' row
group_row = pd.Series(['TYPE'] + ['group'] * (len(metadata_mouse.columns) - 1), index=metadata_mouse.columns)
metadata_mouse = pd.concat([group_row.to_frame().T, metadata_mouse], ignore_index=True)

# Limit DataFrame to the first 12 columns
metadata_mouse = metadata_mouse.iloc[:, :12]
metadata_mouse

,NAME,biosample_id,donor_id,species,species__ontology_label,disease,disease__ontology_label,organ,organ__ontology_label,library_preparation_protocol,library_preparation_protocol__ontology_label,sex
0,TYPE,group,group,group,group,group,group,group,group,group,group,group
1,Z0OYT_AAACGGGCAACACGCCx,CTRLC57AllOther1,1.0,NCBITaxon_10090,Mus musculus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
2,Z0OYT_AAACCTGAGTGTACGGx,CTRLC57AllOther1,1.0,NCBITaxon_10090,Mus musculus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
3,Z0OYT_AAAGTAGGTTGTTTGGx,CTRLC57AllOther1,1.0,NCBITaxon_10090,Mus musculus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
4,Z0OYT_AAAGTAGCATTCCTCGx,CTRLC57AllOther1,1.0,NCBITaxon_10090,Mus musculus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...
8147,H5JEB_TTTACTGTCGGCATCGx,NaN,NaN,NCBITaxon_10090,Mus musculus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
8148,H5JEB_TTTGGTTGTTAAAGACx,NaN,NaN,NCBITaxon_10090,Mus musculus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
8149,H5JEB_TTTATGCGTCTGATTGx,NaN,NaN,NCBITaxon_10090,Mus musculus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
8150,H5JEB_TTTCCTCGTTCAGCGCx,NaN,NaN,NCBITaxon_10090,Mus musculus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown


In [8]:
metadata_mouse = metadata_mouse.drop_duplicates(subset='NAME', keep='first')
metadata_mouse

,NAME,biosample_id,donor_id,species,species__ontology_label,disease,disease__ontology_label,organ,organ__ontology_label,library_preparation_protocol,library_preparation_protocol__ontology_label,sex
0,TYPE,group,group,group,group,group,group,group,group,group,group,group
1,Z0OYT_AAACGGGCAACACGCCx,CTRLC57AllOther1,1.0,NCBITaxon_10090,Mus musculus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
2,Z0OYT_AAACCTGAGTGTACGGx,CTRLC57AllOther1,1.0,NCBITaxon_10090,Mus musculus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
3,Z0OYT_AAAGTAGGTTGTTTGGx,CTRLC57AllOther1,1.0,NCBITaxon_10090,Mus musculus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
4,Z0OYT_AAAGTAGCATTCCTCGx,CTRLC57AllOther1,1.0,NCBITaxon_10090,Mus musculus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...
5551,H5JEB_TTTACTGTCCGCTGTTx,CTRLC57AllOther2,1.0,NCBITaxon_10090,Mus musculus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
5552,H5JEB_TTTATGCGTCTGATTGx,CTRLC57AllOther2,1.0,NCBITaxon_10090,Mus musculus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
5553,H5JEB_TTTCCTCTCTTGCAAGx,CTRLC57AllOther2,1.0,NCBITaxon_10090,Mus musculus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown
5554,H5JEB_TTTCCTCGTTCAGCGCx,CTRLC57AllOther2,1.0,NCBITaxon_10090,Mus musculus,PATO_0000461,normal,UBERON_0000966,retina,EFO_0009899,10X 3' v2,unknown


### Error Checking for Transformation for Species Metadata and UMAP (Clustering) Files

In [9]:
file_path = "/Users/hannahkang/Desktop/shekhar-lab/Single Cell Portal/Output Files/Marmoset/SCP-marmoset-clustering-submission.csv"
clustering_marmoset = pd.read_csv(file_path)

file_path = "/Users/hannahkang/Desktop/shekhar-lab/Single Cell Portal/Output Files/Mouse/SCP-mouse-clustering-submission.csv"
clustering_mouse = pd.read_csv(file_path)

# Example of Clustering File Format
clustering_mouse.head()

,NAME,X,Y
0,TYPE,numeric,numeric
1,Z0OYT_AAACGGGCAACACGCCx,12.2872521763422,10.8380569849402
2,Z0OYT_AAACCTGAGTGTACGGx,6.87598775678653,1.99284834007327
3,Z0OYT_AAAGTAGGTTGTTTGGx,-9.62521959489804,3.70287269691531
4,Z0OYT_AAAGTAGCATTCCTCGx,-7.14581848329525,5.00803083519045


In [10]:
# OBJECTIVE: Check for any cells present within either a cluster or metadata file for a species that do not exist in the other file. 
# In order to be correct, the set() for all test cases should be empty. 

# FOR MARMOSETS:
cell_names_metadata = set(metadata_marmoset['NAME'])
cell_names_cluster = set(clustering_marmoset['NAME'])

cells_not_in_cluster = cell_names_metadata - cell_names_cluster
cells_not_in_metadata = cell_names_cluster - cell_names_metadata

# print("Cells present in metadata_marmoset but not in cluster_file:", cells_not_in_cluster)
# print("Cells present in cluster_file but not in metadata_marmoset:", cells_not_in_metadata)

# FOR MOUSE:
cell_names_metadata = set(metadata_mouse['NAME'])
cell_names_cluster = set(clustering_mouse['NAME'])

cells_not_in_cluster = cell_names_metadata - cell_names_cluster
cells_not_in_metadata = cell_names_cluster - cell_names_metadata

# print("Cells present in metadata_mouse but not in cluster_file:", cells_not_in_cluster)
# print("Cells present in cluster_file but not in metadata_mouse:", cells_not_in_metadata)

if (cells_not_in_cluster == set()) and (cells_not_in_metadata == set()):
    print("ALL PASSED!")

ALL PASSED!


### CellxGene h5ad File Conversion

In [11]:
import scanpy as sc

# os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

# ann_data_2 = sc.read_h5ad("/Users/hannahkang/Desktop/shekhar-lab/CellxGene/MouseBC.h5ad", backed = 'r+')
# ann_data_2.write_h5ad('mouseBC_modified2.h5ad')

In [12]:
# h5clear –-status filename.h5 

# import time
# import h5py
# import scanpy as sc
 
# def open_file_with_retry(filename, mode='r', max_retries=5, retry_delay=1):
#     retries = 0
#     while retries < max_retries:
#         try:
#             with h5py.File(filename, mode) as f:
#                 mouseBC = sc.read_h5ad(filename, backed='r+')
#                 mouseBC.__dict__['_raw'].__dict__['_var'] = mouseBC.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
#                 return mouseBC
#         except IOError as e:
#             if e.errno == 35:
#                 # File is locked, wait and retry
#                 print("File is locked, retrying...")
#                 time.sleep(retry_delay)
#                 retries += 1
#             else:
#                 raise  # If it's a different IOError, raise it

#     # If max retries reached and file still locked, raise an error
#     raise IOError("Max retries reached. Unable to open file.")

# # Usage
# filename = "/Users/hannahkang/Desktop/shekhar-lab/Original Files/MouseBC.h5ad"
# mouseBC = open_file_with_retry(filename)


In [13]:
# Clearing h5ad file (to not have access issues later on)
file_path = '/Users/hannahkang/Desktop/shekhar-lab/CellxGene/Output Files/Mouse/mouseBC_modified_real.h5ad'
with h5py.File(file_path, 'r+') as f:
    try:
        # Check if the dataset/group exists before clearing
        if 'dataset_name' in f:
            del f['dataset_name']  # Delete the dataset/group
            # Or overwrite the dataset/group with empty data
            # f.create_dataset('dataset_name', data=None)
        else:
            print("Dataset or group not found in the H5AD file.")
    except KeyError as e:
        print(f"KeyError: {e}")

Dataset or group not found in the H5AD file.


In [14]:



with h5py.File('/Users/hannahkang/Desktop/shekhar-lab/Original Files/MouseBC.h5ad', 'r+') as f:
    mouseBC = sc.read_h5ad("/Users/hannahkang/Desktop/shekhar-lab/Original Files/MouseBC.h5ad", backed = 'r+')
    mouseBC.__dict__['_raw'].__dict__['_var'] = mouseBC.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
    # mouseBC.write_h5ad("MouseBC_v2.h5ad")



# file_path = '/Users/hannahkang/Desktop/shekhar-lab/Original Files/MouseBC.h5ad'
# with h5py.File(file_path, 'r+') as f:
#     try:
#         # Check if the group and dataset exist
#         if '_raw' in f and '_var' in f['_raw'] and '_index' in f['_raw/_var'].attrs:
#             # Rename the '_index' attribute in the '_var' group
#             f['_raw/_var'].attrs.modify('_index', newname='features')
#         else:
#             print("Group '_raw' or subgroup '_var' or attribute '_index' not found in the H5AD file.")
#     except KeyError as e:
#         print(f"KeyError: {e}")

# # Now, load the modified data with Scanpy
# mouseBC = sc.read_h5ad(file_path, backed='r+')

In [15]:
print(mouseBC.obsm)
print(mouseBC.obsm["X_umap"])

print(' ')
#print(ann_data_marmoset.obsm)
#print(ann_data_marmoset.obsm["X_umap"])

AxisArrays with keys: X_tsne, X_umap
[[ 12.28725218  10.83805698]
 [  6.87598776   1.99284834]
 [ -9.62521959   3.7028727 ]
 ...
 [-10.01020552   3.79835219]
 [ 11.4269874   -2.27397066]
 [ 16.8380134    5.4951424 ]]
 


In [16]:
mouseBC.uns = {"title": "Mouse Bipolar"}


In [17]:
print(mouseBC.obs_keys)

mouseBC.obs["organism_ontology_term_id"] = "NCBITaxon:10090"
mouseBC.obs["tissue_ontology_term_id"] = "UBERON_0000966"
mouseBC.obs["assay_ontology_term_id"] = "EFO:0009809"
mouseBC.obs["disease_ontology_term_id"] = "PATO_0000461"
mouseBC.obs["cell_type_ontology_term_id"] = "UBERON_0000966"
mouseBC.obs["self_reported_ethnicity_ontology_term_id"] = "NA"
mouseBC.obs["development_stage_ontology_term_id"] = "MmusDv"
mouseBC.obs["sex_ontology_term_id"] = "Unknown"
mouseBC.obs["donor_id"] = "animal"
mouseBC.obs["suspension_type"] = "cell"

<bound method AnnData.obs_keys of AnnData object with n_obs × n_vars = 5555 × 31053 backed at '/Users/hannahkang/Desktop/shekhar-lab/Original Files/MouseBC.h5ad'
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'orig.file', 'animal', 'RNA_snn_res.0.5', 'seurat_clusters', 'dendro_order', 'integrated_snn_res.0.8', 'integrated_snn_res.0.5', 'barcode', 'annotated', 'type'
    var: 'features'
    uns: 'title'
    obsm: 'X_tsne', 'X_umap'>


In [18]:
mouseBC.var.rename(columns={'_index': 'features'}, inplace=True)

mouseBC.write_h5ad('/Users/hannahkang/Desktop/shekhar-lab/CellxGene/Output Files/Mouse/mouseBC_modified_real.h5ad')

# ann_data_3 = sc.read_h5ad("mouseBC_modified_real.h5ad", backed = 'r+')

In [19]:
#returns a boolean for each gene if it starts with the letters 'GM' or not
# mouseBC.var['gm'] = mouseBC.var_names.str.startswith('GM')
# mouseBC.var['gm']

In [20]:
# print(mouseBC)
# sc.pp.calculate_qc_metrics(mouseBC, qc_vars=['gm'], inplace=True)

In [21]:
#print(ann_data.X[:10])
#print(ann_data.raw.X[:10])
print(mouseBC.obsm)
print(mouseBC.obsm["X_umap"])

# print(' ')
# print(ann_data_marmoset.obsm)
# print(ann_data_marmoset.obsm["X_umap"])

AxisArrays with keys: X_tsne, X_umap
[[ 12.28725218  10.83805698]
 [  6.87598776   1.99284834]
 [ -9.62521959   3.7028727 ]
 ...
 [-10.01020552   3.79835219]
 [ 11.4269874   -2.27397066]
 [ 16.8380134    5.4951424 ]]


In [22]:
#Fetching the variable names of the h5ad data
mouseBC.var_names
# ann_data_marmoset.var_names

Index(['XKR4', 'GM37381', 'RP1', 'SOX17', 'GM37323', 'MRPL15', 'RGS20',
       'NPBWR1', '4732440D04RIK', 'GM26901',
       ...
       'GM28406', 'GM29436', 'GM28407', 'GM29393', 'GM21294', 'GM28672',
       'GM28670', 'GM29504', 'GM20837', 'GM47283'],
      dtype='object', length=31053)

In [23]:
mouseBC.var_names_make_unique()
# ann_data_marmoset.var_names_make_unique()

In [24]:
mouseBC.obs_names

Index(['possorted_genome_bam_Z0OYT:AAACGGGCAACACGCCx',
       'possorted_genome_bam_Z0OYT:AAACCTGAGTGTACGGx',
       'possorted_genome_bam_Z0OYT:AAAGTAGGTTGTTTGGx',
       'possorted_genome_bam_Z0OYT:AAAGTAGCATTCCTCGx',
       'possorted_genome_bam_Z0OYT:AAATGCCAGCCACTATx',
       'possorted_genome_bam_Z0OYT:AAACGGGTCTTGCCGTx',
       'possorted_genome_bam_Z0OYT:AAAGCAATCAGTACGTx',
       'possorted_genome_bam_Z0OYT:AAACGGGTCAGCAACTx',
       'possorted_genome_bam_Z0OYT:AAACGGGGTAGGCATGx',
       'possorted_genome_bam_Z0OYT:AAAGATGTCTCGATGAx',
       ...
       'possorted_genome_bam_H5JEB:TTTCCTCAGGCTATCTx',
       'possorted_genome_bam_H5JEB:TTTGGTTTCTCCGGTTx',
       'possorted_genome_bam_H5JEB:TTTCCTCAGGTAAACTx',
       'possorted_genome_bam_H5JEB:TTTACTGTCGGCATCGx',
       'possorted_genome_bam_H5JEB:TTTGGTTGTTAAAGACx',
       'possorted_genome_bam_H5JEB:TTTACTGTCCGCTGTTx',
       'possorted_genome_bam_H5JEB:TTTATGCGTCTGATTGx',
       'possorted_genome_bam_H5JEB:TTTCCTCTCTTGCAAGx',

In [25]:
print(mouseBC.X)

# print(ann_data_marmoset.X)

CSRDataset: backend hdf5, shape (5555, 31053), data_dtype float64


In [26]:
#returns a boolean for each gene if it starts with the letters 'GM' or not
# mouseBC.var['gm'] = mouseBC.var_names.str.startswith('GM')
# mouseBC.var['gm']

In [27]:
# print(mouseBC)
# sc.pp.calculate_qc_metrics(mouseBC, qc_vars=['gm'], inplace=True)

print(mouseBC.obs_keys)

print(mouseBC.obs["self_reported_ethnicity_ontology_term_id"])

#Save the modified ann_data file
# mouseBC.write_h5ad('mouseBC_modified.h5ad')

<bound method AnnData.obs_keys of AnnData object with n_obs × n_vars = 5555 × 31053 backed at '/Users/hannahkang/Desktop/shekhar-lab/CellxGene/Output Files/Mouse/mouseBC_modified_real.h5ad'
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'orig.file', 'animal', 'RNA_snn_res.0.5', 'seurat_clusters', 'dendro_order', 'integrated_snn_res.0.8', 'integrated_snn_res.0.5', 'barcode', 'annotated', 'type', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'disease_ontology_term_id', 'cell_type_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'donor_id', 'suspension_type'
    var: 'features'
    uns: 'title'
    obsm: 'X_tsne', 'X_umap'>
possorted_genome_bam_Z0OYT:AAACGGGCAACACGCCx    NA
possorted_genome_bam_Z0OYT:AAACCTGAGTGTACGGx    NA
possorted_genome_bam_Z0OYT:AAAGTAGGTTGTTTGGx    NA
possorted_genome_bam_Z0OYT:AAAGTAGCATTCCTCGx    NA
possorted_genome_bam_Z0OYT:AAATGCCAGCCACTATx    N

In [28]:
# print(ann_data_marmoset.obsm["X_umap"])

In [29]:
# Dario's reading the file

mouseBC = sc.read_h5ad("/Users/hannahkang/Desktop/shekhar-lab/CellxGene/Output Files/Mouse/mouseBC_modified_real.h5ad")
mouseBC

# Example of output: 
# AnnData object with n_obs × n_vars = 5555 × 31053
#     obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'orig.file', 'animal', 'RNA_snn_res.0.5', 'seurat_clusters', 'dendro_order', 'integrated_snn_res.0.8', 'integrated_snn_res.0.5', 'barcode', 'annotated', 'type', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'disease_ontology_term_id', 'cell_type_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'donor_id', 'suspension_type'
#     var: 'features'
#     uns: 'Title'
#     obsm: 'X_tsne', 'X_umap'

AnnData object with n_obs × n_vars = 5555 × 31053
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'orig.file', 'animal', 'RNA_snn_res.0.5', 'seurat_clusters', 'dendro_order', 'integrated_snn_res.0.8', 'integrated_snn_res.0.5', 'barcode', 'annotated', 'type', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'disease_ontology_term_id', 'cell_type_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'donor_id', 'suspension_type'
    var: 'features'
    uns: 'title'
    obsm: 'X_tsne', 'X_umap'

In [30]:
# Marmoset

# Clearing h5ad file (to not have access issues later on)

# file_path = '/Users/hannahkang/Desktop/shekhar-lab/CellxGene/Output Files/Mouse//Users/hannahkang/Desktop/shekhar-lab/CellxGene/Output Files/Marmoset/filtered_ann_data_marmoset.h5ad.h5ad'
# with h5py.File(file_path, 'r+') as f:
#     try:
#         # Check if the dataset/group exists before clearing
#         if 'dataset_name' in f:
#             del f['dataset_name']  # Delete the dataset/group
#             # Or overwrite the dataset/group with empty data
#             # f.create_dataset('dataset_name', data=None)

#         else:
#             print("Dataset or group not found in the H5AD file.")
#     except KeyError as e:
#         print(f"KeyError: {e}")

In [31]:
ann_data_marmoset = sc.read_h5ad("/Users/hannahkang/Desktop/shekhar-lab/Original Files/MarmosetFoveaBC.h5ad", backed = 'r+')

# Convert string-type categorical annotations to categoricals on the full AnnData object
ann_data_marmoset.strings_to_categoricals()

# ann_data_marmoset = ann_data_marmoset.copy(filename='ann_data_marmoset.h5ad')
# Convert string-type categorical annotations to categoricals on the full AnnData object
# ann_data_marmoset.strings_to_categoricals()
# print("Made a copy)")
    

ann_data_marmoset.__dict__['_raw'].__dict__['_var'] = ann_data_marmoset.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
# ann_data_marmoset.write_h5ad("MarmosetBC_v2.h5ad")

print(ann_data_marmoset.obsm)
print(ann_data_marmoset.obsm["X_umap"])

ann_data_marmoset.uns = {"title": "Marmoset Bipolar"}

print(ann_data_marmoset.obs_keys)

# MUST CHECK IF THE DATA IS ACTUALLY CORRECT !!!
ann_data_marmoset.obs["organism_ontology_term_id"] = "NCBITaxon:10090"
ann_data_marmoset.obs["tissue_ontology_term_id"] = "UBERON_0000966"
ann_data_marmoset.obs["assay_ontology_term_id"] = "EFO:0009809"
ann_data_marmoset.obs["disease_ontology_term_id"] = "PATO_0000461"
ann_data_marmoset.obs["cell_type_ontology_term_id"] = "UBERON_0000966"
ann_data_marmoset.obs["self_reported_ethnicity_ontology_term_id"] = "NA"
ann_data_marmoset.obs["development_stage_ontology_term_id"] = "MmusDv"
ann_data_marmoset.obs["sex_ontology_term_id"] = "Unknown"
ann_data_marmoset.obs["donor_id"] = "animal"
ann_data_marmoset.obs["suspension_type"] = "cell"

# ann_data_marmoset.write_h5ad('/Users/hannahkang/Desktop/shekhar-lab/CellxGene/Output Files/Marmoset/ann_data_marmoset_modified_real.h5ad')

# returns a boolean for each gene if it starts with the letters 'GM' or not
# ann_data_marmoset.var['gm'] = ann_data_marmoset.var_names.str.startswith('GM')
# ann_data_marmoset.var['gm']

print(ann_data_marmoset.obsm)
print(ann_data_marmoset.obsm["X_umap"])

#Fetching the variable names of the h5ad data
ann_data_marmoset.var_names

ann_data_marmoset.var_names_make_unique()

ann_data_marmoset.obs_names

ann_data_marmoset.obs.animal

# print(ann_data_marmoset.X)


AxisArrays with keys: X_tsne, X_umap
[[  0.603162     9.92028538]
 [-10.56769328   0.07132263]
 [-10.37739902   1.68855516]
 ...
 [  0.98721249   9.85885254]
 [-10.08541541   1.2832434 ]
 [-10.49356132   1.53761617]]
<bound method AnnData.obs_keys of AnnData object with n_obs × n_vars = 12556 × 27665 backed at '/Users/hannahkang/Desktop/shekhar-lab/Original Files/MarmosetFoveaBC.h5ad'
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'orig.file', 'animal', 'RNA_snn_res.0.5', 'seurat_clusters', 'dendro_order', 'cell_class', 'integrated_snn_res.1', 'integrated_snn_res.0.8', 'all_BC_labels', 'labels', 'type'
    var: 'features'
    uns: 'title'
    obsm: 'X_tsne', 'X_umap'>
AxisArrays with keys: X_tsne, X_umap
[[  0.603162     9.92028538]
 [-10.56769328   0.07132263]
 [-10.37739902   1.68855516]
 ...
 [  0.98721249   9.85885254]
 [-10.08541541   1.2832434 ]
 [-10.49356132   1.53761617]]


possorted_genome_bam_EXKMG:AAAGATGAGCCAGGATx    1.0
possorted_genome_bam_EXKMG:AAAGATGAGGCCCTCAx    1.0
possorted_genome_bam_EXKMG:AAACGGGTCTTGTATCx    1.0
possorted_genome_bam_EXKMG:AAATGCCCACTAGTACx    1.0
possorted_genome_bam_EXKMG:AAACGGGGTCCTAGCGx    1.0
                                               ... 
possorted_genome_bam_K564P:TTTGGTTGTTGAGTTCx    2.0
possorted_genome_bam_K564P:TTTATGCCATGTTCCCx    2.0
possorted_genome_bam_K564P:TTTATGCCAATCACACx    2.0
possorted_genome_bam_K564P:TTTGTCATCTGTTTGTx    2.0
possorted_genome_bam_K564P:TTTATGCCACATGTGTx    2.0
Name: animal, Length: 12556, dtype: float64

In [32]:
# print(ann_data_marmoset.uns['title'])

# print(ann_data_marmoset.obs['orig.ident'].unique())
# print(ann_data_marmoset.obs['nCount_RNA'].unique())
# print(ann_data_marmoset.obs['nFeature_RNA'].unique())
# print(ann_data_marmoset.obs['orig.file'].unique())
# print(ann_data_marmoset.obs['animal'].unique())
# print(ann_data_marmoset.obs['RNA_snn_res.0.5'].unique())
# print(ann_data_marmoset.obs['seurat_clusters'].unique())
# print(ann_data_marmoset.obs['dendro_order'].unique())
# print(ann_data_marmoset.obs['cell_class'].unique())
# print(ann_data_marmoset.obs['integrated_snn_res.1'].unique())
# print(ann_data_marmoset.obs['integrated_snn_res.0.8'].unique())
# print(ann_data_marmoset.obs['all_BC_labels'].unique())
# print(ann_data_marmoset.obs['labels'].unique())
# print(ann_data_marmoset.obs['type'].unique())

# print(ann_data_marmoset.obs_keys())


# print("hello")
# print(ann_data_marmoset.obs['animal'])

print(ann_data_marmoset.var)
print(ann_data_marmoset.var_names)

                  features
LOC108590668  LOC108590668
SLITRK6            SLITRK6
LOC103791423  LOC103791423
LOC108590671  LOC108590671
LOC108588276  LOC108588276
...                    ...
LOC108589989  LOC108589989
LOC103790416  LOC103790416
LOC108589990  LOC108589990
VAMP7                VAMP7
IL9R                  IL9R

[27665 rows x 1 columns]
Index(['LOC108590668', 'SLITRK6', 'LOC103791423', 'LOC108590671',
       'LOC108588276', 'DCT', 'TGDS', 'LOC103795520', 'SOX21', 'ABCC4',
       ...
       'TSPAN3', 'SCAPER', 'LOC108589987', 'LOC103790415', 'ETFA',
       'LOC108589989', 'LOC103790416', 'LOC108589990', 'VAMP7', 'IL9R'],
      dtype='object', length=27665)


In [33]:
ann_data_marmoset.var

,features
LOC108590668,LOC108590668
SLITRK6,SLITRK6
LOC103791423,LOC103791423
LOC108590671,LOC108590671
LOC108588276,LOC108588276
...,...
LOC108589989,LOC108589989
LOC103790416,LOC103790416
LOC108589990,LOC108589990
VAMP7,VAMP7


#### CellxGene: Cutting Down CellxGene to Only Ensembl ID Matching for Marmosets

In [34]:
# EXAMPLE FROM: https://github.com/jrderuiter/pybiomart

dataset = Dataset(name='hsapiens_gene_ensembl',
                  host='http://www.ensembl.org')

dataset.query(attributes=['ensembl_gene_id', 'external_gene_name'],
              filters={'chromosome_name': ['1','2']})

,Gene stable ID,Gene name
0,ENSG00000290825,DDX11L2
1,ENSG00000223972,DDX11L1
2,ENSG00000227232,WASH7P
3,ENSG00000278267,MIR6859-1
4,ENSG00000243485,MIR1302-2HG
...,...,...
10059,ENSG00000291147,NaN
10060,ENSG00000220804,LINC01881
10061,ENSG00000224160,CICP10
10062,ENSG00000244528,SEPTIN14P2


In [35]:
# checking the dataset filters for list of valid filters

dataset = Dataset(name='cjacchus_gene_ensembl', host='http://www.ensembl.org')

# Get the filters
filters = dataset.filters

# Print the filter names and descriptions
for filter_name, description in filters.items():
    print(f"Filter Name: {filter_name}\nDescription: {description}\n")

Filter Name: link_go_closure
Description: <biomart.Filter name='link_go_closure', type='text'>

Filter Name: link_ensembl_transcript_stable_id
Description: <biomart.Filter name='link_ensembl_transcript_stable_id', type='text'>

Filter Name: gene_id
Description: <biomart.Filter name='gene_id', type='text'>

Filter Name: transcript_id
Description: <biomart.Filter name='transcript_id', type='text'>

Filter Name: link_ensembl_gene_id
Description: <biomart.Filter name='link_ensembl_gene_id', type='text'>

Filter Name: chromosome_name
Description: <biomart.Filter name='chromosome_name', type='text'>

Filter Name: start
Description: <biomart.Filter name='start', type='text'>

Filter Name: end
Description: <biomart.Filter name='end', type='text'>

Filter Name: strand
Description: <biomart.Filter name='strand', type='text'>

Filter Name: chromosomal_region
Description: <biomart.Filter name='chromosomal_region', type='text'>

Filter Name: id_list_xrefs_filters
Description: <biomart.Filter name='

In [36]:

# Keep batch size between 600-700. 100 is ~8 min, 600 is ~47 sec, 650 is ~0.3 sec
def convert_gene_names_to_stable_ids(gene_names, batch_size=650): 
    dataset = Dataset(name='cjacchus_gene_ensembl', host='http://www.ensembl.org') # change the dataset name based on the species
    gene_stable_ids = []

    # Split gene names into batches
    for i in range(0, len(gene_names), batch_size):
        gene_batch = gene_names[i:i+batch_size]
        
        # Query Biomart for each batch
        response = dataset.query(attributes=['ensembl_gene_id', 'external_gene_name'], 
                                 filters={'gene_id': list(gene_batch)})
        
        # Map gene names to stable IDs
        gene_map = {row['external_gene_name']: row['Gene stable ID'] for _, row in response.iterrows()}
        gene_stable_ids.extend([gene_map.get(name, 'NaN') for name in gene_batch])

    return gene_stable_ids

countmatrix_marmoset['Gene stable ID'] = convert_gene_names_to_stable_ids(countmatrix_marmoset['Unnamed: 0'])
print(countmatrix_marmoset[['Gene stable ID', 'Unnamed: 0']])

      Gene stable ID    Unnamed: 0
0                NaN  LOC108590668
1                NaN       SLITRK6
2                NaN  LOC103791423
3                NaN  LOC108590671
4                NaN  LOC108588276
...              ...           ...
27660            NaN  LOC108589989
27661            NaN  LOC103790416
27662            NaN  LOC108589990
27663            NaN         VAMP7
27664            NaN          IL9R

[27665 rows x 2 columns]


In [37]:
dataset = Dataset(name='cjacchus_gene_ensembl', host='http://www.ensembl.org')  # Use the correct dataset name for marmosets

# Query Biomart to retrieve a sample of gene names
response = dataset.query(attributes=['external_gene_name'])

print("Sample of gene names from Ensembl Biomart:")
for index, row in response.iterrows():
    print(row['Gene name'])


Sample of gene names from Ensembl Biomart:
U6
U4
SLITRK6
cja-mir-31
PCID2
SLITRK5
PCDH20
GAS1
ATP7B
MIR17
MIR18A
MIR19A
CUL4A
cja-mir-20a
cja-mir-19b-1
cja-mir-92a-2
ZNF484
CARNMT1
GPC5
Metazoa_SRP
LAMP1
RFX3
CDKN2B
Y_RNA
DMRTA1
TMEFF1
GRTP1
GPC6
BAG1
ADPRHL1
PRXL2C
DCT
CAVIN4
TRPM6
CHMP5
GGTA1
DCUN1D2
WDR41
PLPPR1
TGDS
GLIS3
U1
ELAVL2
TMCO3
GPR180
NFX1
BAAT
SLC1A1
TDRD3
TMEM272
COL5A1
PIM3
MRPL50
IZUMO3
PRKRIP1
SPATA6L
USP20
ZNF189
5S_rRNA
RNase_MRP
PLPP6
SH2B2
TUSC1
RORB
CDKL2
ELL2
FCN2
YDJC
ELF2
U7
ANKH
HPS4
LPCAT1
ISOC1
WDFY2
AK3
TFDP1
TBX1
CCDC116
ODAPH
HABP4
C5orf22
FNBP1
CAAP1
ALDOB
SDF2L1
OTULIN
HCN1
RHOBTB3
TLR3
SLC27A6
SRRD
IPO11
DAB2IP
PDE8B
cja-mir-130b
CDC37L1
MOV10L1
AQP3
THAP6
PLAA
TFIP11
JAKMIP2
NOCT
TMED7
SLC6A3
GNB1L
OTULINL
ATP4B
GLRX
ANKRD55
TICAM2
TPST2
AGA
CYFIP2
GRK1
RCHY1
INTS6
PIERCE1
SPATA9
FEM1C
PPIL2
RCL1
MRPS2
CRYBB1
LCN1
RFESD
IFT74
TMEM255B
CRYBA4
GPR150
DIAPH3
NEIL3
NOL6
SLC35D2
ANXA1
PARM1
ABCC4
ARSK
JAK2
PANX2
GPR107
LRRC19
SERPINE3
MRPS30
PGAP4
PRDM5


In [38]:
def check_gene_mismatches(gene_names):
    dataset = Dataset(name='cjacchus_gene_ensembl', host='http://www.ensembl.org')  # Use the correct dataset name for marmosets
    
    # all available gene names
    response = dataset.query(attributes=['external_gene_name'])

    biomart_gene_names = set(response['Gene name'])

    mismatched_gene_names = [name for name in gene_names if name not in biomart_gene_names]

    return mismatched_gene_names

unique_gene_names = countmatrix_marmoset['Unnamed: 0'].unique()

mismatched_genes = check_gene_mismatches(unique_gene_names)

if mismatched_genes:
    print("Mismatched gene names found:")
    for gene in mismatched_genes:
        print(gene)
else:
    print("All gene names in the DataFrame match the gene names in Ensembl Biomart.")


Mismatched gene names found:
LOC108590668
LOC103791423
LOC108590671
LOC108588276
LOC103795520
SOX21
LOC100392244
LOC108590704
LOC103789439
LOC108590733
LOC103791692
LOC108590757
LOC108590777
LOC108590776
LOC108589030
LOC108589034
LOC108589041
LOC108590789
LOC108590794
LOC103792635
LOC100408208
LOC108590798
LOC100413608
LOC108590808
LOC108590810
LOC100384906
LOC108590827
LOC108590720
CCDC168
KDELC1
LOC103793684
LOC103793949
LOC103794074
LOC103794203
FAM155A
LOC108589170
LOC100404462
LOC103794432
LOC103794512
LOC103794541
LOC108590866
LOC103794672
LOC108589187
LOC103794996
LOC103795413
LOC103795419
LOC103795694
LOC103795678
LOC108590940
LOC100415044
LOC108590949
LOC103796016
LOC103796240
LOC103796350
LOC108589297
LOC103796643
LOC103796746
LOC108591015
LOC100402520
LOC108589309
LOC103787766
CLN5
LOC100406506
LOC103788291
LOC103788312
LOC103788360
LOC103788372
LOC108591121
LOC100395014
LOC100394653
LOC108591135
LOC100397189
LOC100407962
LOC108591143
LOC108591145
LOC108591148
LOC100408323
L

In [39]:
def check_gene_matches(gene_names):
    dataset = Dataset(name='cjacchus_gene_ensembl', host='http://www.ensembl.org')  # Use the correct dataset name for marmosets
    
    # all available gene names
    response = dataset.query(attributes=['external_gene_name'])

    biomart_gene_names = set(response['Gene name'])

    matched_gene_names = [name for name in gene_names if name in biomart_gene_names]

    return matched_gene_names

unique_gene_names = countmatrix_marmoset['Unnamed: 0'].unique()

matched_genes = check_gene_matches(unique_gene_names)

print(matched_genes)

# About half are matched & half mismatched 
len(matched_genes)

# Convert matched_genes list to set for faster lookup
matched_genes_set = set(matched_genes)

['SLITRK6', 'DCT', 'TGDS', 'ABCC4', 'DZIP1', 'UGGT2', 'OXGR1', 'RNF113B', 'STK24', 'SLC15A1', 'DOCK9', 'GPR18', 'GPR183', 'GGACT', 'TMTC4', 'NALCN', 'FGF14', 'METTL21C', 'TEX30', 'SLC10A2', 'EFNB2', 'ARGLU1', 'LIG4', 'IRS2', 'COL4A1', 'RAB20', 'CARS2', 'ANKRD10', 'TUBGCP3', 'PCID2', 'GRTP1', 'DCUN1D2', 'ATP4B', 'GAS6', 'RASA3', 'NDFIP2', 'SLAIN1', 'SCEL', 'ACOD1', 'LMO7', 'UCHL3', 'KLF5', 'PIBF1', 'BORA', 'TDRD3', 'PCDH17', 'OLFM4', 'ATP7B', 'WDFY2', 'SERPINE3', 'FAM124A', 'TRIM13', 'ARL11', 'CAB39L', 'CDADC1', 'MLNR', 'FNDC3A', 'RB1', 'ITM2B', 'NUDT15', 'LRCH1', 'LRRC63', 'ZNF484', 'GAS1', 'DIRAS2', 'NFIL3', 'SHC3', 'SEMA4D', 'NOL8', 'OGN', 'OMD', 'ASPN', 'ECM2', 'BICD2', 'NINJ1', 'BARX1', 'ZNF782', 'CTSV', 'FBP2', 'FBP1', 'FANCC', 'PTCH1', 'HSD17B3', 'SLC35D2', 'ZNF658', 'NAA35', 'NTRK2', 'RMI1', 'IDNK', 'TLE4', 'PSAT1', 'CEP78', 'FOXB2', 'OSTF1', 'RORB', 'ANXA1', 'TMC1', 'GDA', 'SMC5', 'MAMDC2', 'PIP5K1B', 'SPTLC1', 'PGM5', 'PUM3', 'RFX3', 'GLIS3', 'SPATA6L', 'AK3', 'INSL6', 'PLGRKT

In [40]:
num_rows_var = ann_data_marmoset.var.shape[0]
print("Original h5ad files gene number of rows: "+ str(num_rows_var))

# Convert filtered_var_names list to set for faster lookup
filtered_var_names_set = set(matched_genes)

# Create a boolean mask indicating whether each gene name in var_names is in filtered_var_names_set
mask = ann_data_marmoset.var_names.isin(filtered_var_names_set)

# Filter the AnnData object based on the boolean mask
ann_data_marmoset = ann_data_marmoset[:, mask].to_memory()
# ann_data_marmoset = (ann_data_marmoset[:, mask]).copy("/Users/hannahkang/Desktop/shekhar-lab/CellxGene/Output Files/Marmoset/filtered_ann_data_marmoset.h5ad")

# Print the filtered AnnData object
print(ann_data_marmoset)


# # Update the 'var' attribute of the AnnData object with the filtered DataFrame
# ann_data_marmoset.var = filtered_var

# # Print the updated 'var' DataFrame
# print(ann_data_marmoset.var)

# # Print the number of rows after filtering
num_rows_var = print(ann_data_marmoset.var.shape[0])
print("Updated h5ad files gene number of rows:", num_rows_var)

Original h5ad files gene number of rows: 27665
AnnData object with n_obs × n_vars = 12556 × 13902
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'orig.file', 'animal', 'RNA_snn_res.0.5', 'seurat_clusters', 'dendro_order', 'cell_class', 'integrated_snn_res.1', 'integrated_snn_res.0.8', 'all_BC_labels', 'labels', 'type', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'disease_ontology_term_id', 'cell_type_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'donor_id', 'suspension_type'
    var: 'features'
    uns: 'title'
    obsm: 'X_tsne', 'X_umap'
13902
Updated h5ad files gene number of rows: None


In [41]:
ann_data_marmoset.var

# filtered_ann_data_marmoset.var

,features
SLITRK6,SLITRK6
DCT,DCT
TGDS,TGDS
ABCC4,ABCC4
DZIP1,DZIP1
...,...
TSPAN3,TSPAN3
SCAPER,SCAPER
ETFA,ETFA
VAMP7,VAMP7


In [42]:
# filtered_ann_data_marmoset
ann_data_marmoset

AnnData object with n_obs × n_vars = 12556 × 13902
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'orig.file', 'animal', 'RNA_snn_res.0.5', 'seurat_clusters', 'dendro_order', 'cell_class', 'integrated_snn_res.1', 'integrated_snn_res.0.8', 'all_BC_labels', 'labels', 'type', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'disease_ontology_term_id', 'cell_type_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'donor_id', 'suspension_type'
    var: 'features'
    uns: 'title'
    obsm: 'X_tsne', 'X_umap'

In [43]:
# filtered_ann_data_marmoset.write_h5ad('/Users/hannahkang/Desktop/shekhar-lab/CellxGene/Output Files/Marmoset/ann_data_marmoset_modified_real.h5ad')

# FILTERED !!
# # Dario's reading the file
# filtered_ann_data_marmoset = sc.read_h5ad("/Users/hannahkang/Desktop/shekhar-lab/CellxGene/Output Files/Marmoset/filtered_ann_data_marmoset.h5ad")
# filtered_ann_data_marmoset  

# should be n_obs × n_vars = 12556 × 13902

original_ann_data_marmoset = sc.read_h5ad("/Users/hannahkang/Desktop/shekhar-lab/Original Files/MarmosetFoveaBC.h5ad")
# filtered_ann_data_marmoset  
original_ann_data_marmoset.strings_to_categoricals()


# # Example of output: 
# # AnnData object with n_obs × n_vars = 5555 × 31053
# #     obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'orig.file', 'animal', 'RNA_snn_res.0.5', 'seurat_clusters', 'dendro_order', 'integrated_snn_res.0.8', 'integrated_snn_res.0.5', 'barcode', 'annotated', 'type', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'disease_ontology_term_id', 'cell_type_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'donor_id', 'suspension_type'
# #     var: 'features'
# #     uns: 'Title'
# #     obsm: 'X_tsne', 'X_umap'

In [44]:
ann_data_marmoset

AnnData object with n_obs × n_vars = 12556 × 13902
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'orig.file', 'animal', 'RNA_snn_res.0.5', 'seurat_clusters', 'dendro_order', 'cell_class', 'integrated_snn_res.1', 'integrated_snn_res.0.8', 'all_BC_labels', 'labels', 'type', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'disease_ontology_term_id', 'cell_type_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'donor_id', 'suspension_type'
    var: 'features'
    uns: 'title'
    obsm: 'X_tsne', 'X_umap'

In [45]:
# Load your full AnnData object (assuming you have already processed it)
# ann_data_marmoset = sc.read_h5ad("/Users/hannahkang/Desktop/shekhar-lab/Original Files/MarmosetFoveaBC.h5ad")

# Convert string-type categorical annotations to categorical data types on the full AnnData object
ann_data_marmoset.strings_to_categoricals()

# ann_data_marmoset

file_path = "/Users/hannahkang/Desktop/shekhar-lab/CellxGene/Output Files/Marmoset/filtered_ann_data_marmoset.h5ad"

ann_data_marmoset.write_h5ad(file_path)


### NeMO Expression Tab Normalized Counts Conversion

In [51]:
# ann_data_marmoset.X

In [56]:
# sc.pp.normalize_total(ann_data_marmoset)
# print(ann_data_marmoset.X[0:11, 0:11])

  (0, 0)	2.707296169730928
  (0, 5)	2.047695371369117
  (1, 5)	1.1507038962226472
  (1, 8)	1.1507038962226472
  (1, 10)	1.6155903052781901
  (2, 10)	1.530574847686639
  (3, 0)	2.2595328884612687
  (5, 10)	1.6073282604104056
  (9, 0)	2.438321978548278
  (10, 0)	2.139383143396472


In [47]:
# Closing h5ad files
del mouseBC 
del ann_data_marmoset
# # del filtered_ann_data_marmoset

In [48]:
# filtered_ann_data_marmoset
# NameError: name 'filtered_ann_data_marmoset' is not defined

### Translate Dataframes to CSV Files for Submission to SCP

In [49]:
file_path = "/Users/hannahkang/Desktop/shekhar-lab/Single Cell Portal/Output Files/Marmoset/SCP-metadata-submission-marmoset.csv"
metadata_marmoset.to_csv(file_path, index=False)

file_path = "/Users/hannahkang/Desktop/shekhar-lab/Single Cell Portal/Output Files/Mouse/SCP-metadata-submission-mouse.csv"
metadata_mouse.to_csv(file_path, index=False)

### Scratch Work Code (Ignore)